In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/recipe/recipes_eda.csv


In [ ]:
from IPython.core.display import display, HTML

## 전체 구간을 넓게
display(HTML("<style>.container { width:100% !important; }</style>"))
## 각 컬럼 width 최대로
pd.set_option('display.max_colwidth', -1)
## rows 500
pd.set_option('display.max_rows', 500)
## columns
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


/tmp/ipykernel_29/1067223850.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/tmp/ipykernel_29/1067223850.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


### 전처리된 레시피 데이터

- 데이터를 68개의 카테고리로 인식

#### Roboflow 이미지 클래스에 맞춰서 전처리 진행

In [ ]:
final_class = [
    "sausage", "bacon", "chicken breast", "chicken thigh", "chicken wing", "beef", "pork", "lamb", "rib", "ham",
    "onion", "garlic", "tomato", "corn", "potato", "ginger", "carrot", "pea", "mushroom", "celery", "spinach",
    "zucchini", "broccoli", "cucumber", "cabbage", "pumpkin", "avocado", "green beans", "radish", "asparagus",
    "leek", "cauliflower", "eggplant", "fennel", "lettuce", "chilli", "sweet potato", "lemon", "apple", "orange",
    "lime", "coconut", "pineapple", "banana", "avocado", "peach", "pear", "apricot", "strawberry", "mango",
    "grapefruit", "watermelon", "plantain", "tangerine", "cream", "chocolate", "yogurt", "sugar", "butter",
    "cheese", "egg", "flour", "milk", "rice", "tofu", "noodle", "oil", "salt"
]

In [ ]:
roboflow_class= [
    'apple', 'apricot', 'asparagus', 'avocado', 'bacon', 'banana', 'bean', 'beef', 'blackberry',
    'blueberry', 'bread', 'broccoli', 'butter', 'cabbage', 'candy', 'carrot', 'cashew', 'cauliflower',
    'celery', 'cheese', 'chicken breast', 'chicken thigh', 'chicken wing', 'chicken', 'chili', 'chilli',
    'chocolate', 'coconut', 'cookie', 'corn', 'cream', 'cucumber', 'egg', 'eggplant', 'fennel', 'fig',
    'fish', 'flour', 'garlic', 'ginger', 'grape', 'grapefruit', 'ham', 'icecream', 'kiwi', 'lamb', 'leek',
    'lemon', 'lettuce', 'lime', 'mango', 'meatball', 'milk', 'mushroom', 'noodle', 'noodles', 'oil',
    'onion', 'orange', 'pea', 'peach', 'pear', 'pepper', 'pineapple', 'pork', 'potato', 'pumpkin', 'radish',
    'rib', 'rice', 'salmon', 'salt', 'sausage', 'spinach', 'spring onion', 'strawberry', 'sugar',
    'sweet potato', 'tofu', 'tomato', 'watermelon', 'yogurt', 'zucchini'
]

#### final_class 클래스에는 있지만 Roboflow 에는 없는값

In [ ]:
not_robo_classes = [fc for fc in final_class if not(fc in roboflow_class)]
print(not_robo_classes)

['green beans', 'plantain', 'tangerine']


#### Roboflow 클래스에는 있지만 final_class 에는 없는값

In [ ]:
not_final_classes = [robo for robo in roboflow_class if not(robo in final_class)]
print(not_final_classes)

['bean', 'blackberry', 'blueberry', 'bread', 'candy', 'cashew', 'chicken', 'chili', 'cookie', 'fig', 'fish', 'grape', 'icecream', 'kiwi', 'meatball', 'noodles', 'pepper', 'salmon', 'spring onion']


In [ ]:
df = pd.read_csv('/kaggle/input/recipe/recipes_eda.csv')
df.head()

,name,id,minutes,n_steps,steps,description,n_ingredients,calories,total fat,sugar,sodium,protein,saturated fat,carbohydrates,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,vege/vegan,food type,healthiness,ingredients
0,arriba baked winter squash mexican style,137739,55,11,"['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt']","autumn is my favorite time of year to cook! this recipe \r\ncan be prepared either spicy or sweet, your choice!\r\ntwo of my posted mexican-inspired seasoning mix recipes are offered as suggestions.",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0,winter squash,mexican seasoning,mixed spice,honey,butter,oil,salt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dish,NaN,"['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']"
1,a bit different breakfast pizza,31490,30,9,"['preheat oven to 425 degrees f', 'press dough into the bottom and sides of a 12 inch pizza pan', 'bake for 5 minutes until set but not browned', 'cut sausage into small pieces', 'whisk eggs and milk in a bowl until frothy', 'spoon sausage over baked crust and sprinkle with cheese', 'pour egg mixture slowly over sausage and cheese', 's& p to taste', 'bake 15-20 minutes or until eggs are set and crust is brown']",this recipe calls for the crust to be prebaked a bit before adding ingredients. feel free to change sausage to ham or bacon. this warms well in the microwave for those late risers.,6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,prepared pizza crust,sausage,egg,milk,salt,cheese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dish,NaN,"['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']"
2,all in the kitchen chili,112140,130,6,"['brown ground beef in large pot', 'add chopped onions to ground beef when almost brown and sautee until wilted', 'add all other ingredients', 'add kidney beans if you like beans in your chili', 'cook in slow cooker on high for 2-3 hours or 6-8 hours on low', 'serve with cold clean lettuce and shredded cheese']",this modified version of 'mom's' chili was a hit at our 2004 christmas party. we made an extra large pot to have some left to freeze but it never made it to the freezer. it was a favorite by all. perfect for any cold and rainy day. you won't find this one in a cookbook. it is truly an original.,13,269.8,22.0,32.0,48.0,39.0,27.0,5.0,beef,onion,tomatoes,tomato paste,soup,tomatoes,kidney beans,water,chili powder,cumin,salt,lettuce,cheese,NaN,NaN,NaN,dish,NaN,"['ground beef', 'yellow onions', 'diced tomatoes', 'tomato paste', 'tomato soup', 'rotel tomatoes', 'kidney beans', 'water', 'chili powder', 'ground cumin', 'salt', 'lettuce', 'cheddar cheese']"
3,alouette potatoes,59389,45,11,"['place potatoes in a large pot of lightly salted water and bring to a gentle boil', 'cook until potatoes are just tender', 'drain', 'place potatoes in a large bowl and add all ingredients except the""alouette""', 'mix well and transfer to a buttered 8x8 inch glass baking dish with 2 inch sides', 'press the potatoes with a spatula to make top as flat as possible', 'set aside for 2 hours at room temperature', 'preheat oven to 350^f', 'spread""alouette"" evenly over potatoes and bake 15 minutes', 'divide between plates', 'garnish with finely diced red and yellow bell peppers']","this is a super easy, great tasting, make ahead side dish that looks l

In [ ]:
df.columns

Index(['name', 'id', 'minutes', 'n_steps', 'steps', 'description', 'n_ingredients', 'calories', 'total fat', 'sugar', 'sodium', 'protein', 'saturated fat', 'carbohydrates', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', 'vege/vegan', 'food type', 'healthiness', 'ingredients'], dtype='object')

In [ ]:
cols = ['name', 'ingredients', 'vege/vegan', 'food type', 'healthiness']
cols.extend(map(str,list(range(0, 15))))
data = df[cols]
data

,name,ingredients,vege/vegan,food type,healthiness,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,arriba baked winter squash mexican style,"['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']",NaN,dish,NaN,winter squash,mexican seasoning,mixed spice,honey,butter,oil,salt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a bit different breakfast pizza,"['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']",NaN,dish,NaN,prepared pizza crust,sausage,egg,milk,salt,cheese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,all in the kitchen chili,"['ground beef', 'yellow onions', 'diced tomatoes', 'tomato paste', 'tomato soup', 'rotel tomatoes', 'kidney beans', 'water', 'chili powder', 'ground cumin', 'salt', 'lettuce', 'cheddar cheese']",NaN,dish,NaN,beef,onion,tomatoes,tomato paste,soup,tomatoes,kidney beans,water,chili powder,cumin,salt,lettuce,cheese,NaN,NaN
3,alouette potatoes,"['spreadable cheese with garlic and herbs', 'new potatoes', 'shallots', 'parsley', 'tarragon', 'olive oil', 'red wine vinegar', 'salt', 'pepper', 'red bell pepper', 'yellow bell pepper']",NaN,dish,healthy,garlic,potatoes,shallots,parsley,tarragon,oil,vinegar,salt,pepper,pepper,pepper,NaN,NaN,NaN,NaN
4,amish tomato ketchup for canning,"['tomato juice', 'apple cider vinegar', 'sugar', 'salt', 'pepper', 'clove oil', 'cinnamon oil', 'dry mustard']",NaN,dish,healthy,tomato juice,vinegar,sugar,salt,pepper,oil,oil,mustard,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230181,zydeco soup,"['celery', 'onion', 'green sweet pepper', 'garlic cloves', 'olive oil', 'cooked ham', 'paprika', 'sugar', 'dry mustard', 'ground cumin', 'dried basil', 'dried oregano', 'dried thyme', 'ground cloves', 'black pepper', 'cayenne pepper', 'black-eyed peas', 'yellow hominy', 'diced tomatoes', 'low sodium chicken broth', 'fresh parsley', 'molasses']",NaN,dish,healthy,celery,onion,pepper,garlic,oil,cooked ham,paprika,sugar,mustard,cumin,basil,oregano,thyme,cloves,pepper
230182,zydeco spice mix,"['paprika', 'salt', 'garlic powder', 'onion powder', 'dried basil', 'dried oregano', 'dried tarragon', 'dried thyme', 'powdered sugar', 'black pepper', 'cayenne pepper', 'red pepper flakes', 'celery seed']",NaN,dish,NaN,paprika,salt,garlic,onion,basil,oregano,tarragon,thyme,sugar,pepper,pepper,pepper,celery seed,NaN,NaN
230183,zydeco ya ya deviled eggs,"['hard-cooked eggs', 'mayonnaise', 'dijon mustard', 'salt-free cajun seasoning', 'tabasco sauce', 'salt', 'black pepper', 'fresh italian parsley']",NaN,dish,NaN,egg,mayonnaise,dijon mustard,salt,tabasco sauce,salt,pepper,italian parsley,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230184,cookies by design cookies on a stick,"['butter', 'eagle brand condensed milk', 'light brown sugar', 'sour cream', 'egg', 'extract', 'nutmeg', 'self-rising flour', 'bisquick', 'wooden popsicle sticks']",NaN,dessert,NaN,butter,milk,sugar,sour cream,egg,extract,nutmeg,flour,bisquick,wooden popsicle sticks,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['food type'].value_counts()

dish       194885
dessert    35301 
Name: food type, dtype: int64

In [ ]:
df['healthiness'].value_counts()

healthy    120131
Name: healthiness, dtype: int64

In [ ]:
df['vege/vegan'].value_counts()

vegan         250
vegetarian    237
Name: vege/vegan, dtype: int64

### new_ingredients 컬럼 생성
- 전처리된 재료 리스트 생성

In [ ]:
df2 = pd.DataFrame(data)

In [ ]:
df2['new_ingredients'] = df2.apply(lambda row: row.iloc[5:].dropna().tolist(), axis=1)

cols = ['name', 'ingredients', 'vege/vegan', 'food type', 'healthiness', 'new_ingredients']
cols.extend(map(str, list(range(0, 15))))
df2 = df2.reindex(columns=cols)
df2.head()

,name,ingredients,vege/vegan,food type,healthiness,new_ingredients,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,arriba baked winter squash mexican style,"['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']",NaN,dish,NaN,"[winter squash, mexican seasoning, mixed spice, honey, butter, oil, salt]",winter squash,mexican seasoning,mixed spice,honey,butter,oil,salt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a bit different breakfast pizza,"['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']",NaN,dish,NaN,"[prepared pizza crust, sausage, egg, milk, salt, cheese]",prepared pizza crust,sausage,egg,milk,salt,cheese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,all in the kitchen chili,"['ground beef', 'yellow onions', 'diced tomatoes', 'tomato paste', 'tomato soup', 'rotel tomatoes', 'kidney beans', 'water', 'chili powder', 'ground cumin', 'salt', 'lettuce', 'cheddar cheese']",NaN,dish,NaN,"[beef, onion, tomatoes, tomato paste, soup, tomatoes, kidney beans, water, chili powder, cumin, salt, lettuce, cheese]",beef,onion,tomatoes,tomato paste,soup,tomatoes,kidney beans,water,chili powder,cumin,salt,lettuce,cheese,NaN,NaN
3,alouette potatoes,"['spreadable cheese with garlic and herbs', 'new potatoes', 'shallots', 'parsley', 'tarragon', 'olive oil', 'red wine vinegar', 'salt', 'pepper', 'red bell pepper', 'yellow bell pepper']",NaN,dish,healthy,"[garlic, potatoes, shallots, parsley, tarragon, oil, vinegar, salt, pepper, pepper, pepper]",garlic,potatoes,shallots,parsley,tarragon,oil,vinegar,salt,pepper,pepper,pepper,NaN,NaN,NaN,NaN
4,amish tomato ketchup for canning,"['tomato juice', 'apple cider vinegar', 'sugar', 'salt', 'pepper', 'clove oil', 'cinnamon oil', 'dry mustard']",NaN,dish,healthy,"[tomato juice, vinegar, sugar, salt, pepper, oil, oil, mustard]",tomato juice,vinegar,sugar,salt,pepper,oil,oil,mustard,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### new_ingredient 중복값 제거  (pepper, pepper,,,,)

In [ ]:
df2['new_ingredients'] = df2['new_ingredients'].apply(lambda x: list(set(x)))

In [ ]:
df2.iloc[3,:]

name               alouette  potatoes                                                                                                                                                                        
ingredients        ['spreadable cheese with garlic and herbs', 'new potatoes', 'shallots', 'parsley', 'tarragon', 'olive oil', 'red wine vinegar', 'salt', 'pepper', 'red bell pepper', 'yellow bell pepper']
vege/vegan         NaN                                                                                                                                                                                       
food type          dish                                                                                                                                                                                      
healthiness        healthy                                                                                                                                                      

In [ ]:
df_filter_ingredients = df2[['name', 'new_ingredients']]
df_filter_ingredients

,name,new_ingredients
0,arriba baked winter squash mexican style,"[oil, mexican seasoning, salt, mixed spice, butter, winter squash, honey]"
1,a bit different breakfast pizza,"[sausage, milk, cheese, prepared pizza crust, egg, salt]"
2,all in the kitchen chili,"[onion, tomato paste, lettuce, beef, cheese, kidney beans, chili powder, cumin, salt, tomatoes, soup, water]"
3,alouette potatoes,"[vinegar, pepper, oil, tarragon, salt, shallots, garlic, parsley, potatoes]"
4,amish tomato ketchup for canning,"[vinegar, pepper, oil, tomato juice, salt, sugar, mustard]"
...,...,...
230181,zydeco soup,"[ thyme, onion, pepper, oil, cooked ham, cumin, basil, sugar, oregano, mustard, cloves, celery, garlic, paprika]"
230182,zydeco spice mix,"[ thyme, onion, pepper, salt, basil, sugar, tarragon, oregano, celery seed, garlic, paprika]"
230183,zydeco ya ya deviled eggs,"[pepper, tabasco sauce, mayonnaise, egg, salt, dijon mustard, italian parsley]"
230184,cookies by design cookies on a stick,"[milk, egg, sugar, sour cream, extract, flour, wooden popsicle sticks, butter, nutmeg, bisquick]"


### Roboflow 포함 재료와 아닌 재료 파악
- class_ingredient : 유연한 기준으로 roboflow 클래스에 포함되는 단어를 포함한 식재료를 수집한 카테고리
- other_ingredient : roboflow 클래스에 존재하지 않는 단어들로 구성된 카테고리


In [ ]:
def filter_ingredients_in_class(ingredient_list):
    return [ingredient for ingredient in ingredient_list if any(robo in ingredient for robo in roboflow_class)]

df_filter_ingredients['class_ingredient'] = df2['new_ingredients'].apply(filter_ingredients_in_class)


/tmp/ipykernel_29/3943617318.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter_ingredients['class_ingredient'] = df2['new_ingredients'].apply(filter_ingredients_in_class)


In [ ]:
def filter_ingredients_other_class(ingredient_list):
    return [ingredient for ingredient in ingredient_list if not any(robo in ingredient for robo in roboflow_class)]

# Apply the filter function to the 'ingredients' column
df_filter_ingredients['other_ingredient'] = df2['new_ingredients'].apply(filter_ingredients_other_class)


/tmp/ipykernel_29/2238464657.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter_ingredients['other_ingredient'] = df2['new_ingredients'].apply(filter_ingredients_other_class)


In [ ]:
df_filter_ingredients.head()

,name,new_ingredients,class_ingredient,other_ingredient
0,arriba baked winter squash mexican style,"[oil, mexican seasoning, salt, mixed spice, butter, winter squash, honey]","[oil, salt, butter]","[mexican seasoning, mixed spice, winter squash, honey]"
1,a bit different breakfast pizza,"[sausage, milk, cheese, prepared pizza crust, egg, salt]","[sausage, milk, cheese, egg, salt]",[prepared pizza crust]
2,all in the kitchen chili,"[onion, tomato paste, lettuce, beef, cheese, kidney beans, chili powder, cumin, salt, tomatoes, soup, water]","[onion, tomato paste, lettuce, beef, cheese, kidney beans, chili powder, salt, tomatoes]","[ cumin, soup, water]"
3,alouette potatoes,"[vinegar, pepper, oil, tarragon, salt, shallots, garlic, parsley, potatoes]","[pepper, oil, salt, garlic, potatoes]","[vinegar, tarragon, shallots, parsley]"
4,amish tomato ketchup for canning,"[vinegar, pepper, oil, tomato juice, salt, sugar, mustard]","[pepper, oil, tomato juice, salt, sugar]","[vinegar, mustard]"


### other_ingredient 분석
- other_ingredient 를  중복없는 하나의 리스트로 만들어 살펴본다


In [ ]:
from itertools import chain
from collections import Counter
import ast

# Combine all the lists from 'new_ingredients' into a single list
combined_list = list(chain.from_iterable(df_filter_ingredients['other_ingredient']))

# Remove duplicates by converting the list to a set and then back to a list
unique_other_list = list(set(combined_list))


print(len(unique_other_list))
unique_other_list[:10]


6280


['pecan pie',
 'martha white wild berry muffin mix',
 'brown lentils',
 'scallion s',
 'steak seasoning',
 'creme de menthe coffee syrup',
 'cracked bulgur wheat',
 'baby kai lan',
 'soy whip',
 'vita brits']

In [ ]:
df_filter_ingredients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230186 entries, 0 to 230185
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   name              230185 non-null  object
 1   new_ingredients   230186 non-null  object
 2   class_ingredient  230186 non-null  object
 3   other_ingredient  230186 non-null  object
dtypes: object(4)
memory usage: 7.0+ MB


In [ ]:
# Combine all the lists from 'new_ingredients' into a single list
combined_list = list(chain.from_iterable(df2['new_ingredients']))

# Remove duplicates by converting the list to a set and then back to a list
unique_ingredients_list = list(set(combined_list))


print(len(unique_ingredients_list))
unique_ingredients_list[:10]

9565


['pecan pie',
 'martha white wild berry muffin mix',
 'leek mix',
 'brown lentils',
 'scallion s',
 'steak seasoning',
 'green beans',
 'creme de menthe coffee syrup',
 'cracked bulgur wheat',
 'radish tops']

In [ ]:
roboflow_class[:20]

['apple',
 'apricot',
 'asparagus',
 'avocado',
 'bacon',
 'banana',
 'bean',
 'beef',
 'blackberry',
 'blueberry',
 'bread',
 'broccoli',
 'butter',
 'cabbage',
 'candy',
 'carrot',
 'cashew',
 'cauliflower',
 'celery',
 'cheese']

### roboflow class 에 관련 재료만 있는 레시피만 추출  - 12460

#### roblflow class 식재료 단어를 포함하는 식재료면 다 포함시킴 (apple juice - apple, cream - ice cream)

In [ ]:
# 데이터의 개수가 0인 행 추출
flex_robo_df = df_filter_ingredients[df_filter_ingredients['other_ingredient'].apply(len) == 0]
flex_robo_df = flex_robo_df[['name', 'new_ingredients', 'other_ingredient']]
flex_robo_df = flex_robo_df.reset_index(drop=True)
flex_robo_df.drop('other_ingredient', axis=1, inplace=True)
flex_robo_df

,name,new_ingredients
0,apple a day milk shake,"[milk, apple, frozen apple juice concentrate, vanilla ice cream]"
1,bananas 4 ice cream pie,"[chocolate sandwich style cookies, whipped cream, chocolate syrup, strawberry ice cream, bananas, vanilla ice cream]"
2,chile rellenos,"[oil, cheese, egg, whole green chilies, cornstarch]"
3,chinese candy,"[noodle, salt, butter]"
4,easiest ever hollandaise sauce,"[pepper, egg, salt, lemon, juice of, butter]"
...,...,...
12456,zucchini cheddar casserole,"[cheese, zucchini, egg, sour cream, flour, breadcrumbs, butter, bacon]"
12457,zucchini potato pancakes,"[onion, pepper, oil, zucchini, egg, salt, flour, sour cream, potato]"
12458,zucchini sausage brunch casserole,"[sausage, onion, pepper, cheese, zucchini, egg, salt, garlic, italian bread, half-and-half cream]"
12459,zucchini cottage cheese casserole,"[onion, pepper, cheese, oil, zucchini, egg, salt, butter, soft breadcrumbs, rice]"


#### 2. roboflow class의 식재료만 사용하는 레시피 - 2851 개


In [ ]:
def filter_only_robo(ingredient_list):
    return [ingredient for ingredient in ingredient_list if any(robo == ingredient for robo in roboflow_class)]

In [ ]:
only_robo_df = df2[df2['new_ingredients'].apply(lambda x: all(ingredient in roboflow_class for ingredient in x))]
only_robo_df = only_robo_df[['name', 'new_ingredients']]
only_robo_df = only_robo_df.reset_index(drop=True)
only_robo_df

,name,new_ingredients
0,chinese candy,"[noodle, salt, butter]"
1,the elvis smoothie,"[banana, milk, sugar, butter]"
2,no name beef recipe loco moco,"[onion, rice, egg, beef]"
3,the creamed dried beef chipped beef recipe,"[milk, pepper, beef, flour, butter]"
4,creamed spinach,"[yogurt, onion, spinach, cheese]"
...,...,...
2846,zucchini with bacon corn peppers,"[onion, pepper, cheese, zucchini, corn, bacon, garlic]"
2847,zucchini with green peppercorns lemon and parmesan,"[pepper, lemon, oil, cheese, zucchini, salt]"
2848,zucchini with jalapeno monterey jack,"[flour, zucchini, pepper]"
2849,zucchini with parmesan,"[pepper, cheese, oil, zucchini, salt]"


 시소러스

유의어 사이의 관계를 그래프로 정의하고 있는 방대한 데이터

WordNet을 이용하여 유사한 단어를 파악할 수 있고, 각 단어의 유사도를 계산할 수 있습니다

In [ ]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/


Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [ ]:
import nltk
nltk.download('wordnet')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
!python -m spacy download en_core_web_md


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [ ]:
import spacy
import numpy as np

nlp = spacy.load("en_core_web_md")

def get_embedding(word):
    return nlp(word).vector

def find_most_similar_class(ingredient, classes, threshold=0.9):
    print(ingredient)
    print('------------')
    ingredient_embedding = get_embedding(ingredient)
    max_similarity = -1
    most_similar_class = None

    for class_name in classes:
        class_embedding = get_embedding(class_name)
        similarity = ingredient_embedding.dot(class_embedding)
        if similarity > max_similarity and similarity >= threshold:
            print(similarity, class_name )
            max_similarity = similarity
            most_similar_class = class_name



    return most_similar_class

def replace_with_most_similar_class(ingredients_list, classes, threshold=0.7):
    transformed_list = []

    for ingredient in ingredients_list:
        most_similar_class = find_most_similar_class(ingredient, classes, threshold)
        transformed_list.append(most_similar_class)

    return transformed_list

sample = [
    'apple juice', 'pineapple vodka', 'dessert apples', 'tart apples', 'apple cinnamon cheerios',
    'tart baking apples', 'sweetened crushed pineapple', 'sour apple schnapps', ' apple rings',
    'pippin apples', 'pineapple-mango jam', 'pineapple-orange juice concentrate',
    'pineapple slices in juice', 'pineapple jelly', 'yellow apples', 'braeburn apple', 'applesauce',
    'cortland apples', 'orange-pineapple jell-o', 'crisp red apples', 'pineapple flavoring',
    'pineapple passion sorbet',
]


# Set the similarity threshold (0.7)
similarity_threshold = 0.9

transformed_ingredients = replace_with_most_similar_class(sample, roboflow_class, similarity_threshold)
print(transformed_ingredients)


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

apple juice
------------
1542.0374 apple
pineapple vodka
------------
766.2201 apple
770.19464 cheese
807.8321 cream
865.19885 lemon
981.8725 milk
dessert apples
------------
1101.9406 apple
1129.335 bread
tart apples
------------
1316.019 apple
apple cinnamon cheerios
------------
939.32916 apple
tart baking apples
------------
1217.7438 apple
1340.1776 bread
sweetened crushed pineapple
------------
778.6657 apple
815.76636 bread
843.08997 corn
874.8709 cream
905.61304 lime
913.6158 milk
1102.8745 oil
sour apple schnapps
------------
1136.1501 apple
1147.8297 milk
 apple rings
------------
822.6665 apple
pippin apples
------------
640.62805 apple
660.9617 pork
pineapple-mango jam
------------
695.60376 apple
759.0833 bean
783.35126 cream
pineapple-orange juice concentrate
------------
705.758 apple
785.39545 cream
1129.7365 oil
pineapple slices in juice
------------
753.861 apple
821.9397 bread
869.90314 milk
1209.1844 oil
pineapple jelly
------------
845.09015 apple
923.1912 bean
946

In [ ]:
similar = pd.DataFrame({'ingredient' : sample, 'class' : transformed_ingredients})
similar

,ingredient,class
0,apple juice,apple
1,pineapple vodka,milk
2,dessert apples,bread
3,tart apples,apple
4,apple cinnamon cheerios,apple
5,tart baking apples,bread
6,sweetened crushed pineapple,oil
7,sour apple schnapps,milk
8,apple rings,apple
9,pippin apples,pork


### 데이터 베이스 생성

In [ ]:
# 모든 재료들을 추출하여 중복된 재료 제거
all_ingredients = list(set([ingredient for ingredients in only_robo_df['new_ingredients'] for ingredient in ingredients]))

# 모든 재료들을 컬럼으로 갖는 새로운 데이터프레임 생성
ingredient_df = pd.DataFrame(columns=all_ingredients)

# 각 요리에 대해 해당 재료가 포함되는지 여부에 따라 1 또는 0을 할당
for i, ingredients in enumerate(only_robo_df['new_ingredients']):
    for ingredient in all_ingredients:
        if ingredient in ingredients:
            ingredient_df.at[i, ingredient] = 1
        else:
            ingredient_df.at[i, ingredient] = 0

result_df = pd.concat([only_robo_df['name'], ingredient_df], axis=1)


In [ ]:
result_df

,name,tofu,fennel,ham,pork,rib,chicken thigh,strawberry,salmon,potato,garlic,pumpkin,avocado,chicken wing,mushroom,meatball,lettuce,chili,watermelon,spinach,bread,corn,apple,lime,cabbage,chocolate,bacon,banana,asparagus,chicken breast,chicken,lamb,sausage,leek,grapefruit,onion,peach,lemon,cheese,oil,apricot,cucumber,yogurt,cream,kiwi,egg,pineapple,sugar,butter,carrot,radish,pear,rice,cauliflower,milk,pepper,beef,zucchini,broccoli,blackberry,noodle,salt,flour,mango,coconut,fish,celery,ginger,orange
0,chinese candy,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
1,the elvis smoothie,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,no name beef recipe loco moco,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,the creamed dried beef chipped beef recipe,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0
4,creamed spinach,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2846,zucchini with bacon corn peppers,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2847,zucchini with green peppercorns lemon and parmesan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0
2848,zucchini with jalapeno monterey jack,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
2849,zucchini with parmesan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0
